In [4]:
# melissa robertson February 2022
# UCSF - no price data
# NY Presbyterian - done
# Cedars Sinai - no DRG data
#nyu-langone



# Dependencies
import pandas as pd
import numpy as np
import json
import os
import csv
import re


In [5]:
################################################################################################################

In [6]:
def categorise(row):
    if  'BCBS'.lower() in row['Plan'].lower():
        return 'BCBS'
    elif  'Blue'.lower() in row['Plan'].lower():
        return 'BCBS'
    elif 'Cigna'.lower() in row['Plan'].lower():
        return 'Cigna'
    elif 'Aetna'.lower() in row['Plan'].lower():
        return 'Aetna'
    elif 'Cash'.lower() in row['Plan'].lower():
        return 'Self Pay'
    elif 'Gross Charge'.lower() in row['Plan'].lower():
        return'Gross Charge'
    elif 'United'.lower() in row['Plan'].lower():
        return 'United Healthcare'
    else:
        return 'Other'

In [121]:
#read in UCSF data
filepath = os.path.join("../943281657_ucsf-medical-center_standardcharges.json")
with open(filepath) as jsonfile:
    UCSF_json = pd.read_json(jsonfile)

In [122]:
#subset to DRG only
UCSF_drg=UCSF_json[UCSF_json['Code Type'] =='DRG']

In [123]:
#remove unneccissary columsn
UCSF_drg=UCSF_drg.drop(['Revenue Code','CPT/HCPCS','Default Modifier','CPT/HCPCS Numeric','IP De-Identified Minimum Negotiated Charge','IP De-Identified Maximum Negotiated Charge','OP De-Identified Minimum Negotiated Charge',
       'OP De-Identified Maximum Negotiated Charge'], axis=1)

In [124]:
UCSF_drg.columns

Index(['Code', 'Code Type', 'Description', 'Default Gross Charge',
       'Emergency Care Gross Charge', 'Default Gross Clinic Charge',
       'Hospital Ancillary Global Component Gross Charge',
       'Hospital Based Professional Services Gross Charge',
       'US/MANAGED CHOICE AETNA  IP Negotiated Charge',
       'US/ELECT CHOICE AETNA  IP Negotiated Charge',
       ...
       'TRICARE WEST ACTIVE HNET FED SVCS  OP Negotiated Charge',
       'APPLE INC UHC CHOICE PLUS  OP Negotiated Charge',
       'ADMIN UNITED HLTHCR  OP Negotiated Charge',
       'CHOICE PLUS UNITED HLTHCR  OP Negotiated Charge',
       'ADMIN UNITED HLTHCR (UMR)  OP Negotiated Charge',
       'UNITED HLTHCR SELECT/ PLUS  OP Negotiated Charge',
       'UNITED HLTHCR SELECT/ PLUS POS  OP Negotiated Charge',
       'HILL PHYS MED GRP/SAC WHA  OP Negotiated Charge',
       'SF MIA OP Negotiated Charge',
       'ANTHEM BLUE CONNECTION EPO ANTHEM OP Negotiated Charge'],
      dtype='object', length=210)

In [125]:
#rename columns
UCSF_drg=UCSF_drg.rename(columns={"Code": "DRG Number", "Description": "DRG Description",'Code Type':'Billing Code Type'}) 

In [127]:
#check to see if any charges
UCSF_drg=UCSF_drg.melt(id_vars=['DRG Number', 'Billing Code Type', 'DRG Description'], 
        var_name="Plan", 
        value_name="Charge Amount")

In [110]:
UCSF_drg['Plan'].unique()


array(['Default Gross Charge', 'Emergency Care Gross Charge',
       'Default Gross Clinic Charge',
       'Hospital Ancillary Global Component Gross Charge',
       'Hospital Based Professional Services Gross Charge',
       'IP De-Identified Minimum Negotiated Charge',
       'IP De-Identified Maximum Negotiated Charge',
       'OP De-Identified Minimum Negotiated Charge',
       'OP De-Identified Maximum Negotiated Charge',
       'US/MANAGED CHOICE AETNA  IP Negotiated Charge',
       'US/ELECT CHOICE AETNA  IP Negotiated Charge',
       'US HLTH/OPEN CHOICE AETNA  IP Negotiated Charge',
       'BX PRUDENT BUYER ANTHEM  IP Negotiated Charge',
       'BX CA ANTHEM  IP Negotiated Charge',
       'BX/BS OUT OF STATE BLUE CARD PROGRAM  IP Negotiated Charge',
       'FEDERAL EMPLOYEES/BS ANTHEM  IP Negotiated Charge',
       'PERSCHOICE/PERSCARE ANTHEM  IP Negotiated Charge',
       'GOOGLE HEALTHCARE ANTHEM  IP Negotiated Charge',
       'DELTA HLTH SYSTEMS ANTHEM  IP Negotiated Charge

In [128]:
# create payer column
UCSF_drg['Payor'] = UCSF_drg.apply(lambda row: categorise(row), axis=1)

In [129]:
UCSF_drg['Payor'].unique()

array(['Gross Charge', 'Other', 'Aetna', 'BCBS', 'Cigna',
       'United Healthcare'], dtype=object)

In [130]:
#add additonal columns
UCSF_drg["US News Ranking"]=9
UCSF_drg["Hospital Name"]="UCSF Medical Center"

In [134]:
# reorder columns
UCSF_drg = UCSF_drg[["US News Ranking","Hospital Name", "Billing Code Type","DRG Number", "DRG Description","Payor","Plan","Charge Amount"]]

In [135]:
# ouput csv file
UCSF_drg.to_csv("UCSF Medical Center.csv", index=False)

In [ ]:
#NO charges available for UCSF

In [ ]:
######################################################################################################################

In [ ]:
## NY presbyterian file
#########################################################################################################################

In [10]:
#NYpresbyterian
NYpres_json=pd.read_json("../133957095_NewYorkPresbyterianHospital_updated01132022.json",lines=True,orient='records')


In [11]:
NYpres_json.shape

(2, 5757)

In [12]:
NYpres_json_long=NYpres_json.T
NYpres_json_long[0]

0       {'Code (CPT/DRG)': 1110000001, 'Description': ...
1       {'Code (CPT/DRG)': 1110000004, 'Description': ...
2       {'Code (CPT/DRG)': 1110000005, 'Description': ...
3       {'Code (CPT/DRG)': 1110000006, 'Description': ...
4       {'Code (CPT/DRG)': 1110000007, 'Description': ...
                              ...                        
5752    {'Code (CPT/DRG)': 982, 'Description': 'EXTENS...
5753    {'Code (CPT/DRG)': 983, 'Description': 'EXTENS...
5754    {'Code (CPT/DRG)': 987, 'Description': 'NON-EX...
5755    {'Code (CPT/DRG)': 988, 'Description': 'NON-EX...
5756    {'Code (CPT/DRG)': 989, 'Description': 'NON-EX...
Name: 0, Length: 5757, dtype: object

In [13]:
NYpresdict=[]
i=0
j=0
while j <2:
    for each in NYpres_json_long[j]:
        NYpresdict.append(each)
       # print(each)
        i+=1
    j+=1
    
    

In [14]:
#keep only DRG
NYpresdrg_dict=[]
for each in NYpresdict:
    try:
        if each['Code (CPT/DRG)']<1000:
            NYpresdrg_dict.append(each)
    except:
        continue
     
        

In [16]:
len(NYpresdrg_dict)


756

In [75]:
NYpresdf = pd.DataFrame.from_dict(NYpresdrg_dict)

In [76]:
NYpresdf.columns

Index(['Code (CPT/DRG)', 'Description', 'Rev Code', 'Inpatient/Outpatient',
       'Gross Charges', 'Discounted Cash Price', 'Aetna', 'Cigna', 'Empire',
       'Emblem', 'United', 'Aetna Medicare', 'AgeWell Medicare',
       'Empire Medicare', 'Fidelis Medicare', 'Emblem Medicare',
       'Healthfirst Medicare', 'UHC Community Plan/United Medicare',
       'VNS Medicare', 'WellCare Medicare', '1199',
       'Affinity Molina Medicaid/CHP', 'Affinity Molina Essential',
       'Amida Care Medicaid', 'Empire Healthplus Medicaid',
       'Empire Healthplus Essential', 'Empire Healthplus Exchange',
       'Fidelis Medicaid/CHP', 'Fidelis Essential/Exchange',
       'Healthfirst Essential/Exchange', 'Healthfirst Medicaid/CHP',
       'Emblem Medicaid/CHP', 'MVP Medicaid/CHP', 'MVP Essential',
       'United Commuity Plan Medicaid', 'United Commuity Plan Essential',
       'VNS Medicaid', 'Minimum Negotiated Charge',
       'Maximum Negotiated Charge'],
      dtype='object')

In [77]:
#remove unneccissary columsn
NYpresdf=NYpresdf.drop(["Rev Code","Inpatient/Outpatient",'Minimum Negotiated Charge','Maximum Negotiated Charge'], axis=1)

In [78]:
NYpresdf.columns

Index(['Code (CPT/DRG)', 'Description', 'Gross Charges',
       'Discounted Cash Price', 'Aetna', 'Cigna', 'Empire', 'Emblem', 'United',
       'Aetna Medicare', 'AgeWell Medicare', 'Empire Medicare',
       'Fidelis Medicare', 'Emblem Medicare', 'Healthfirst Medicare',
       'UHC Community Plan/United Medicare', 'VNS Medicare',
       'WellCare Medicare', '1199', 'Affinity Molina Medicaid/CHP',
       'Affinity Molina Essential', 'Amida Care Medicaid',
       'Empire Healthplus Medicaid', 'Empire Healthplus Essential',
       'Empire Healthplus Exchange', 'Fidelis Medicaid/CHP',
       'Fidelis Essential/Exchange', 'Healthfirst Essential/Exchange',
       'Healthfirst Medicaid/CHP', 'Emblem Medicaid/CHP', 'MVP Medicaid/CHP',
       'MVP Essential', 'United Commuity Plan Medicaid',
       'United Commuity Plan Essential', 'VNS Medicaid'],
      dtype='object')

In [79]:
#rename columns
NYpresdf=NYpresdf.rename(columns={"Code (CPT/DRG)": "DRG Number", "Description": "DRG Description",}) 

In [80]:
# convert columns to rows
NYpresdf=NYpresdf.melt(id_vars=["DRG Number", "DRG Description"], 
        var_name="Plan", 
        value_name="Charge Amount")

In [81]:
#add additonal columns
NYpresdf["US News Ranking"]=7
NYpresdf["Hospital Name"]="New York-Presbyterian Hospital"
NYpresdf["Billing Code Type"]="MSDRG"

In [82]:
NYpresdf[NYpresdf['Plan'].str.contains('Cigna')]

,DRG Number,DRG Description,Plan,Charge Amount,US News Ranking,Hospital Name,Billing Code Type
2268,1,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Cigna,1024812.054,7,New York-Presbyterian Hospital,MSDRG
2269,2,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Cigna,1024812.054,7,New York-Presbyterian Hospital,MSDRG
2270,3,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Cigna,746260.830,7,New York-Presbyterian Hospital,MSDRG
2271,4,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Cigna,465692.850,7,New York-Presbyterian Hospital,MSDRG
2272,5,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,Cigna,544148.325,7,New York-Presbyterian Hospital,MSDRG
...,...,...,...,...,...,...,...
3019,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Cigna,99079.596,7,New York-Presbyterian Hospital,MSDRG
3020,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Cigna,64538.838,7,New York-Presbyterian Hospital,MSDRG
3021,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Cigna,127956.654,7,New York-Presbyterian Hospital,MSDRG
3022,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Cigna,66651.984,7,New York-Presbyterian Hospital,MSDRG


In [115]:
# create payer column
NYpresdf['Payor'] = NYpresdf.apply(lambda row: categorise(row), axis=1)

In [116]:
#check that it worked
NYpresdf['Payor'].unique()

array(['Gross Charge', 'Self Pay', 'Aetna', 'Cigna', 'Other',
       'United Healthcare'], dtype=object)

In [86]:
NYpresdf = NYpresdf[["US News Ranking","Hospital Name", "Billing Code Type","DRG Number", "DRG Description","Payor","Plan","Charge Amount"]]

In [88]:
# ouput csv file
NYpresdf.to_csv('New York-Presbyterian.csv', index=False)

In [ ]:
#########################################################################################################################

In [ ]:
#read in Cedars Sinai data;

In [ ]:
##########################################################################################################################

In [136]:
file_path = "../951644600_cedars-sinai-medical-center_standardcharges.csv"
cedars_sinai_df=pd.read_csv(file_path)

In [137]:
cedars_sinai_df.columns

Index(['Inpatient/Outpatient', 'CHARGE_CODE', 'CHARGE_DESC', 'MEDICATION_NAME',
       'ERX_ID', 'NDC_CODE', 'CPT_CODE', 'GROSS_CHARGES', 'MIN', 'MAX',
       'Aetna_HMO', 'Aetna_PPO', 'Blue_Cross_HMO_PPO', 'Blue_Cross_SR_HMO',
       'Blue_Shield_HMO_PPO', 'Blue_Shield_Sr_HMO', 'Cigna_PPO',
       'Healthnet_HMO', 'Healthnet_PPO', 'Kaiser', 'Kaiser_SR_MMC',
       'Multiplan', 'UHC_HMO', 'UHC_PPO', 'UHC_SR_HMO/PPO',
       'Uninsured rate for patients that do not qualify for financial assistance*'],
      dtype='object')

In [138]:
cedars_sinai_df['Inpatient/Outpatient'].unique()

array(['CSMC IP', 'CSMC OPED',
       "*Cedars-Sinai does not have a standard cash price for inpatient services because each patient's specific medical and financial situation is different. If you are uninsured, the price you will pay for inpatient care at Cedars-Sinai will depend on the services you need and the amount of financial assistance you qualify for under our general financial assistance policy.   Although there are no standard cash prices for inpatient services, if you are uninsured and would like information on rates for services, please contact Patient Financial Services at 866-803-1777. We're committed to providing quality healthcare to everyone in the community who needs it. This includes offering full financial assistance or services at discounted prices to eligible people who cannot afford to pay for part or all of their care. Payment plans are also available."],
      dtype=object)

In [139]:
inpatient_df=cedars_sinai_df[cedars_sinai_df['Inpatient/Outpatient']=='CSMC IP']

In [140]:
inpatient_df['CHARGE_CODE'].unique()

array([ 1200001.,  1200002.,  1200003., ...,  9900515.,  9900517.,
       80016217.])

In [142]:
inpatient_df[inpatient_df['CHARGE_CODE']<1000]

,Inpatient/Outpatient,CHARGE_CODE,CHARGE_DESC,MEDICATION_NAME,ERX_ID,NDC_CODE,CPT_CODE,GROSS_CHARGES,MIN,MAX,...,Cigna_PPO,Healthnet_HMO,Healthnet_PPO,Kaiser,Kaiser_SR_MMC,Multiplan,UHC_HMO,UHC_PPO,UHC_SR_HMO/PPO,Uninsured rate for patients that do not qualify for financial assistance*


In [144]:
inpatient_df[inpatient_df['CHARGE_DESC'].str.contains('DRG')]

,Inpatient/Outpatient,CHARGE_CODE,CHARGE_DESC,MEDICATION_NAME,ERX_ID,NDC_CODE,CPT_CODE,GROSS_CHARGES,MIN,MAX,...,Cigna_PPO,Healthnet_HMO,Healthnet_PPO,Kaiser,Kaiser_SR_MMC,Multiplan,UHC_HMO,UHC_PPO,UHC_SR_HMO/PPO,Uninsured rate for patients that do not qualify for financial assistance*
5529,CSMC IP,3091649.0,"HB LAB HIV PHENOTYPING, ADDL DRGS",NaN,NaN,NaN,87904,42.0,3.0,17.0,...,15.0,15.0,15.0,15.0,13.0,17.0,15.0,14.0,4.0,No standard uninsured rate
5600,CSMC IP,3092008.0,HB LAB HIV PHENO INFECT AG DRG,NaN,NaN,NaN,87900,202.0,12.0,81.0,...,70.0,70.0,70.0,69.0,59.0,81.0,72.0,66.0,18.0,No standard uninsured rate
5940,CSMC IP,3092877.0,HB PRTRT RBC W/CHEM OR DRG ARC,NaN,NaN,NaN,86970,42.0,3.0,17.0,...,15.0,15.0,15.0,15.0,13.0,17.0,15.0,14.0,4.0,No standard uninsured rate
5997,CSMC IP,3093040.0,HB DRGQ DRUG SCREEN MULT CLASS,NaN,NaN,NaN,G0480,62.0,4.0,25.0,...,22.0,22.0,22.0,22.0,19.0,25.0,22.0,21.0,6.0,No standard uninsured rate
6035,CSMC IP,3093125.0,HB INF AGNT DRG SUS PH GN BI,NaN,NaN,NaN,87900,233.0,14.0,93.0,...,81.0,81.0,81.0,80.0,68.0,93.0,82.0,76.0,21.0,No standard uninsured rate
7442,CSMC IP,3611219.0,HB CONVERSION EXT BIL DRG CATH,NaN,NaN,NaN,47535,8261.0,599.0,3587.0,...,2842.0,2815.0,2846.0,2639.0,2117.0,2707.0,2173.0,2381.0,663.0,No standard uninsured rate
7443,CSMC IP,3611220.0,HB EXCHANGE BILIARY DRG CATH,NaN,NaN,NaN,47536,8261.0,599.0,2923.0,...,2842.0,2815.0,2846.0,2639.0,2117.0,2707.0,2173.0,2381.0,663.0,No standard uninsured rate
7444,CSMC IP,3611221.0,HB REMOVAL BILIARY DRG CATH,NaN,NaN,NaN,47537,4220.0,306.0,1507.0,...,1452.0,1438.0,1454.0,1349.0,1082.0,1383.0,1110.0,1216.0,339.0,No standard uninsured rate


In [ ]:
#####################################################################################################################
#cedars_sinai doesnt have DRG codes
######################################################################################################################

In [ ]:
#####################################################################################################################
# clean NYU Langone Health files
######################################################################################################################

In [7]:
files=np.arange(13)
filestart="../133971298_nyu-langone-health_standardcharges_"
filenames=[]

In [8]:
###########################################
#read in all 12 files from NYU langone and keep only DRG rows
for item in files:
    name=f"NYU_{item}"
    try: 
         exec(f'NYU_{item}=pd.read_csv(f"{filestart}{item}.csv", low_memory=False)')
         exec(f"NYU_drg_{item}=NYU_{item}[NYU_{item}[' Billing Code'].str.contains('MSDRG', na=False)]") 
         #exec(f"print(NYU_drg_{item}.shape)")
         exec(f"filenames.append(f'NYU_drg_{item}')")
    except: 
        continue

In [9]:
filenames

['NYU_drg_1',
 'NYU_drg_2',
 'NYU_drg_3',
 'NYU_drg_4',
 'NYU_drg_5',
 'NYU_drg_6',
 'NYU_drg_7',
 'NYU_drg_8',
 'NYU_drg_9',
 'NYU_drg_10',
 'NYU_drg_11',
 'NYU_drg_12']

In [10]:
NYU_drg_1.columns

Index(['Identifier Code', ' Billing Code', 'Identifier Description',
       'Hospital A-C Gross Charges', 'Hospital D Gross Charges',
       'Hospital A-C Discounted Cash Price',
       'Hospital D Discounted Cash Price', 'Hospital A1005AETNA HMO',
       'Hospital B1005AETNA HMO', 'Hospital C1005AETNA HMO',
       ...
       'Hospital A1921HEALTHFIRST ABSOLUTE CARE FIDA',
       'Hospital B1921HEALTHFIRST ABSOLUTE CARE FIDA',
       'Hospital C1921HEALTHFIRST ABSOLUTE CARE FIDA',
       'Hospital D1921HEALTHFIRST ABSOLUTE CARE FIDA',
       'Hospital A2010BCBS HEALTHPLUS ESSENTIAL PLAN 1',
       'Hospital B2010BCBS HEALTHPLUS ESSENTIAL PLAN 1',
       'Hospital C2010BCBS HEALTHPLUS ESSENTIAL PLAN 1',
       'Hospital D2010BCBS HEALTHPLUS ESSENTIAL PLAN 1',
       'De-identified Minimum', 'De-identified Maximum'],
      dtype='object', length=151)

In [11]:
NYU_drg_12.shape

(767, 69)

In [12]:
newfilenames=[]
column_names=[]
for file in filenames:
    new_cols=[]
    for column in eval(file):
        if column not in column_names:
          #  print(column)
            column_names.append(column)
            new_cols.append(column)
    try:
        exec(f"new_{file} = {file}[new_cols]")
        exec(f"newfilenames.append(f'new_{file}')")
    except:
        print(f"error {file}")

In [13]:
frames=[new_NYU_drg_1,
new_NYU_drg_2,
new_NYU_drg_3,
new_NYU_drg_4,
new_NYU_drg_5,
new_NYU_drg_6,
new_NYU_drg_7,
new_NYU_drg_8,
new_NYU_drg_9,
new_NYU_drg_10,
new_NYU_drg_11,
new_NYU_drg_12]
result = pd.concat(frames,axis=1)
#result = pd.concat([NYU_drg_1, NYU_drg_2, NYU_drg_3, NYU_drg_4,NYU_drg_5,NYU_drg_6,NYU_drg_7, NYU_drg_8, NYU_drg_9, NYU_drg_10, NYU_drg_11, NYU_drg_12], axis=1)

In [14]:
result.reset_index(drop=True)

,Identifier Code,Billing Code,Identifier Description,Hospital A-C Gross Charges,Hospital D Gross Charges,Hospital A-C Discounted Cash Price,Hospital D Discounted Cash Price,Hospital A1005AETNA HMO,Hospital B1005AETNA HMO,Hospital C1005AETNA HMO,...,Hospital C1802 HEALTHFIRST SILVER LEAF EXCHANGE,Hospital D1802 HEALTHFIRST SILVER LEAF EXCHANGE,"Hospital A2236 HEALTHFIRST BRONZE TOTAL, PRO, PLUS EPO","Hospital B2236 HEALTHFIRST BRONZE TOTAL, PRO, PLUS EPO","Hospital C2236 HEALTHFIRST BRONZE TOTAL, PRO, PLUS EPO","Hospital D2236 HEALTHFIRST BRONZE TOTAL, PRO, PLUS EPO",Hospital A1088 HEALTHNET PPO,Hospital B1088 HEALTHNET PPO,Hospital C1088 HEALTHNET PPO,Hospital D1088 HEALTHNET PPO
0,NaN,MSDRG001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,NaN,NaN,NaN,NaN,618308.78,618308.78,475737.79,...,NaN,NaN,NaN,NaN,NaN,NaN,1446378.33,1446378.33,1446378.33,1446378.33
1,NaN,MSDRG002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,NaN,NaN,NaN,NaN,320135.59,320135.59,246318.03,...,NaN,NaN,NaN,NaN,NaN,NaN,964493.8,964493.8,964493.8,964493.8
2,NaN,MSDRG003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,NaN,NaN,NaN,NaN,408571.12,408571.12,314361.9,...,NaN,NaN,NaN,NaN,NaN,NaN,1498818.88,1498818.88,1498818.88,1498818.88
3,NaN,MSDRG004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,NaN,NaN,NaN,NaN,254962.66,254962.66,196172.82,...,NaN,NaN,NaN,NaN,NaN,NaN,1032038.66,1032038.66,1032038.66,1032038.66
4,NaN,MSDRG005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,NaN,NaN,NaN,NaN,218875.48,218875.48,168406.69,...,NaN,NaN,NaN,NaN,NaN,NaN,1182433.42,1182433.42,1182433.42,1182433.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,NaN,MSDRG987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,NaN,NaN,NaN,NaN,70055.12,70055.12,53901.66,...,NaN,NaN,NaN,NaN,NaN,NaN,250077.71,250077.71,250077.71,250077.71
763,NaN,MSDRG988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,NaN,NaN,NaN,NaN,36491.36,36491.36,28077.11,...,NaN,NaN,NaN,NaN,NaN,NaN,95395.13,95395.13,95395.13,95395.13
764,NaN,MSDRG989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,NaN,NaN,NaN,NaN,24028.19,24028.19,18487.71,...,NaN,NaN,NaN,NaN,NaN,NaN,59943.72,59943.72,59943.72,59943.72
765,NaN,MSDRG998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,32385.98,32385.98,32385.98,32385.98


In [15]:
#create columns

result["DRG Number"]=result[' Billing Code'].str[5:]
result["Billing Code Type"]='MSDRG'

In [16]:
result=result.drop(['Identifier Code',' Billing Code'], axis=1)

In [17]:
#rename columns
result =result.rename(columns={"Identifier Description": "DRG Description"}) 


In [18]:
# convert columns to rows
result=result.melt(id_vars=["DRG Number", "DRG Description",'Billing Code Type'], 
        var_name="Plan", 
        value_name="Charge Amount")

In [19]:
result.drop_duplicates(inplace=True,ignore_index=True)

In [20]:
result

,DRG Number,DRG Description,Billing Code Type,Plan,Charge Amount
0,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,MSDRG,Hospital A-C Gross Charges,NaN
1,002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,MSDRG,Hospital A-C Gross Charges,NaN
2,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,MSDRG,Hospital A-C Gross Charges,NaN
3,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,MSDRG,Hospital A-C Gross Charges,NaN
4,005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,MSDRG,Hospital A-C Gross Charges,NaN
...,...,...,...,...,...
1240234,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,MSDRG,Hospital D1088 HEALTHNET PPO,250077.71
1240235,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,MSDRG,Hospital D1088 HEALTHNET PPO,95395.13
1240236,989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,MSDRG,Hospital D1088 HEALTHNET PPO,59943.72
1240237,998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,MSDRG,Hospital D1088 HEALTHNET PPO,32385.98


In [21]:
# add columns
result["US News Ranking"]=8
result["Hospital Name"]="NYU Langone Hospitals "+ result['Plan'].str[:10]


In [22]:
result

,DRG Number,DRG Description,Billing Code Type,Plan,Charge Amount,US News Ranking,Hospital Name
0,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,MSDRG,Hospital A-C Gross Charges,NaN,8,NYU Langone Hospitals Hospital A
1,002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,MSDRG,Hospital A-C Gross Charges,NaN,8,NYU Langone Hospitals Hospital A
2,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,MSDRG,Hospital A-C Gross Charges,NaN,8,NYU Langone Hospitals Hospital A
3,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,MSDRG,Hospital A-C Gross Charges,NaN,8,NYU Langone Hospitals Hospital A
4,005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,MSDRG,Hospital A-C Gross Charges,NaN,8,NYU Langone Hospitals Hospital A
...,...,...,...,...,...,...,...
1240234,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,MSDRG,Hospital D1088 HEALTHNET PPO,250077.71,8,NYU Langone Hospitals Hospital D
1240235,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,MSDRG,Hospital D1088 HEALTHNET PPO,95395.13,8,NYU Langone Hospitals Hospital D
1240236,989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,MSDRG,Hospital D1088 HEALTHNET PPO,59943.72,8,NYU Langone Hospitals Hospital D
1240237,998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,MSDRG,Hospital D1088 HEALTHNET PPO,32385.98,8,NYU Langone Hospitals Hospital D


In [23]:
# create payer column
result['Payor'] = result.apply(lambda row: categorise(row), axis=1)

In [24]:
result['Payor'].value_counts()

Other                889720
BCBS                 217828
United Healthcare     49855
Cigna                 39884
Aetna                 36816
Gross Charge           3068
Self Pay               3068
Name: Payor, dtype: int64

In [25]:
#Reorder columns
result = result[["US News Ranking","Hospital Name", "Billing Code Type","DRG Number", "DRG Description","Payor","Plan","Charge Amount"]]

In [26]:
result

,US News Ranking,Hospital Name,Billing Code Type,DRG Number,DRG Description,Payor,Plan,Charge Amount
0,8,NYU Langone Hospitals Hospital A,MSDRG,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Gross Charge,Hospital A-C Gross Charges,NaN
1,8,NYU Langone Hospitals Hospital A,MSDRG,002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,Gross Charge,Hospital A-C Gross Charges,NaN
2,8,NYU Langone Hospitals Hospital A,MSDRG,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,Gross Charge,Hospital A-C Gross Charges,NaN
3,8,NYU Langone Hospitals Hospital A,MSDRG,004,TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DI...,Gross Charge,Hospital A-C Gross Charges,NaN
4,8,NYU Langone Hospitals Hospital A,MSDRG,005,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,Gross Charge,Hospital A-C Gross Charges,NaN
...,...,...,...,...,...,...,...,...
1240234,8,NYU Langone Hospitals Hospital D,MSDRG,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Other,Hospital D1088 HEALTHNET PPO,250077.71
1240235,8,NYU Langone Hospitals Hospital D,MSDRG,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Other,Hospital D1088 HEALTHNET PPO,95395.13
1240236,8,NYU Langone Hospitals Hospital D,MSDRG,989,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Other,Hospital D1088 HEALTHNET PPO,59943.72
1240237,8,NYU Langone Hospitals Hospital D,MSDRG,998,PRINCIPAL DIAGNOSIS INVALID AS DISCHARGE DIAGN...,Other,Hospital D1088 HEALTHNET PPO,32385.98


In [27]:
# ouput csv file
result.to_csv('../NYU Langone Hospitals.csv', index=False)